# Registering an XGBoost model on Verta

Within Verta, a "Model" can be any arbitrary function: a traditional ML model (e.g., sklearn, PyTorch, TF, etc); a function (e.g., squaring a number, making a DB function etc.); or a mixture of the above (e.g., pre-processing code, a DB call, and then a model application.) See more [here](https://docs.verta.ai/verta/registry/concepts).

This notebook provides an example of how to register a XGBoost model on Verta as a Verta Standard Model either via  convenience functions or by extending [VertaModelBase](https://verta.readthedocs.io/en/master/_autogen/verta.registry.VertaModelBase.html?highlight=VertaModelBase#verta.registry.VertaModelBase).

<a href="https://colab.research.google.com/github/VertaAI/modeldb/blob/master/client/workflows/examples/xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Imports

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import itertools
import time

import six

import numpy as np
import pandas as pd

import sklearn
from sklearn import datasets
from sklearn import model_selection

import xgboost as xgb

### 0.1 Verta import and setup

In [2]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

In [3]:
import os

# Ensure credentials are set up, if not, use below
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 
# os.environ['VERTA_HOST'] = 

from verta import Client
import os
client = Client(os.environ['VERTA_HOST'])

---

## 1. Model training

### 1.1 Prepare Data

In [4]:
data = datasets.load_wine()

X = data['data']
y = data['target']

dtrain = xgb.DMatrix(X, label=y)

In [5]:
df = pd.DataFrame(np.hstack((X, y.reshape(-1, 1))),
                  columns=data['feature_names'] + ['species'])

df.head()

### 1.2 Train the model

In [6]:
hyperparams =  {
    'eta': 0.5,
    'max_depth' : 2,
    'num_class' : 10
}

model = xgb.XGBClassifier(**hyperparams)
model.fit(X, y)

In [7]:
# Calculate and Log Accuracy on Full Training Set
train_acc = model.score(X, y)
print("Training accuracy: {:.4f}".format(train_acc))

## 2. Register Model for Deployment

In [8]:
registered_model = client.get_or_create_registered_model(
    name="wine", labels=["xgboost"])

In [9]:
from verta.environment import Python
model_version = registered_model.create_standard_model_from_xgboost(
    model, environment=Python(requirements=["xgboost", "sklearn"]), name="v1")

---